# Intro to Decision Trees
Authors: David Yerrington, Matt Brems

<img src="./images/sms1.png" align="left">

### Learning Objectives

- Understand the intuition behind decision trees.
- Calculate Gini.
- Describe how decision trees use Gini to make decisions.
- Fit, generate predictions from, and evaluate decision tree models.
- Interpret and tune `max_depth`, `min_samples_split`, `min_samples_leaf`.

## What will we get for dinner?

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Sushi   |      Sunny     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |

<details><summary>It's a rainy day. Based on our past orders, what do you think we'll order?</summary>

- Indian food.
- In 100% of past cases where the weather is rainy, we've eaten Indian food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Indian  |      Rainy     |   Weekday  |
|   Indian  |      Rainy     |   Weekend  |
|   Indian  |      Rainy     |   Weekday  |

</details>

<details><summary>It's a sunny day. Based on our past orders, what do you think we'll order?</summary>

- Either Sushi or Mexican food... but we can't say with certainty whether we'd eat sushi or Mexican food.
- Based on our past orders, we eat sushi on 1/3 of sunny days and we eat Mexican food on 2/3 of sunny days.
- If I **had** to make a guess here, I'd probably predict Mexican food, but we may want to use additional information to be certain.

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Sushi   |      Sunny     |   Weekday  |
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

<details><summary>It's a sunny day that also happens to be a weekend. Based on our past orders, what do you think we'll order?</summary>

- Mexican food.
- In 100% of past cases where the weather is sunny and where it's a weekend, we've eaten Mexican food!

|$Y = $ Food|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|  Mexican  |      Sunny     |   Weekend  |
|  Mexican  |      Sunny     |   Weekend  |

</details>

# Decision Trees: Overview

A decision tree:
- takes a dataset consisting of $X$ and $Y$ data, 
- finds rules based on our $X$ data that partitions (splits) our data into smaller datasets such that
- by the bottom of the tree, the values $Y$ in each "leaf node" are as "pure" as possible.

We frequently see decision trees represented by a graph.

<img src="./images/order_food_dt.png" alt="order_food" width="750"/>

- (This image was created using [Draw.io](https://www.draw.io/).)

### Terminology
Decision trees look like upside down trees. 
- What we see on top is known as the "root node," through which all of our observations are passed.
- At each internal split, our dataset is partitioned.
- A "parent" node is split into two or more "child" nodes.
- At each of the "leaf nodes" (colored orange), we contain a subset of records that are as pure as possible.
    - In this food example, each leaf node is perfectly pure. Once we get to a leaf node, every observation in that leaf node has the exact same value of $Y$!
    - There are ways to quantify the idea of "purity" here so that we can let our computer do most of the tree-building (model-fitting) process... we'll come back to this later.

Decision trees are also called "**Classification and Regression Trees**," sometimes abbreviated "**CART**."
- [DecisionTreeClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [DecisionTreeRegressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

## 20 Questions

If you aren't familiar with the game [20 Questions](https://en.wikipedia.org/wiki/Twenty_Questions), it's a game with two players (or teams). 
- Player A thinks of an item but doesn't say what the item is.
- Player B then attempts to guess what the item is by asking a series of 20 questions with a yes or no answer.
- If player B correctly guesses the item, then player B wins!
- If player B does not correctly guess the item, then player A wins!

Let's play a quick game of "5 Questions" to get a feel for it.

---



#### Decision trees operate in a fashion that's pretty similar to 20 Questions.
- Decisions are made in a sequential fashion. Once you know a piece of information, you use that piece of information when asking future questions.
    - Example: If you know that the item you're trying to guess is a person, then you can use that information to ask better subsequent questions.
- It's possible to get lucky by making very specific guesses early, but it's pretty unlikely that this is a winning strategy.
    - Example: If you asked, "Is it an airplane? Is it a boat? Is it a car?" as your first three questions, it's not very likely that you'll win the game.

When fitting a decision tree, we're effectively getting our computer to play a game of 20 Questions. We give the computer some data and it figures out the best $X$ variable to split on at the right time.
- Above, our "what food should we order?" decision tree first asked what the weather was, **then** asked whether it was a weekday or weekend.
- If we had asked "is it a weekday or weekend" first, we'd have ended up with a slightly more complicated decision tree.

Just like with all of our models, in order for the computer to learn which $X$ variable to split on and when, the computer needs a loss function to quantify how good a particular split is. This is where the idea of **purity** comes into play.

## Purity in Decision Trees

When quantifying how "pure" a node is, we want to see what the distribution of $Y$ is in each node, then summarize this distribution with a number.

<img src="./images/order_food_dt.png" alt="order_food" width="750"/>

- For continuous $Y$ (i.e. using a decision tree to predict income), the default option is mean squared error.
    - This is the `criterion = 'mse'` argument in `DecisionTreeRegressor`.
    - When the decision tree is figuring out which split to make at a given node, it picks the split that maximizes the drop in MSE from the parent node to the child node.
    

- For discrete $Y$, the default option is the Gini impurity. *(Bonus: This is not quite the same thing as the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).)*

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
\text{Gini impurity (2 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
\text{Gini impurity (3 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 - P(\text{class 3})^2 \\
\end{eqnarray*}
$$

In [3]:
# Create our y variable from our "where should we eat" dataframe.
y = ['Indian', 'Sushi', 'Indian', 'Mexican', 'Indian', 'Mexican']

In [1]:
# Define Gini function, called gini.
def gini(obs):
    
    # Create a list to store my squared class probabilities.
    gini_sum = []
    
    # Iterate through each class.
    for class_i in set(obs):
        
        # Calculate observed probability of class i.
        prob = (obs.count(class_i) / len(obs))
        
        # Square the probability and append it to gini_sum.
        gini_sum.append(prob ** 2)
        
    # Return Gini impurity.
    return 1 - sum(gini_sum)

In [4]:
# Check to see if your Gini function is correct on the 
# "where should we eat" data. (Should get 0.6111.)
gini(y)

0.6111111111111112

### Gini Practice

<details><summary>What is the Gini impurity of a node when every item is from the same class?</summary>
    
- Our Gini impurity is zero.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 \\
&=& 1 - 1^2 \\
&=& 1 - 1 \\
&=& 0
\end{eqnarray*}
$$
</details>

In [5]:
# What is Gini when every item is from the same class?
gini(['Indian', 'Indian', 'Indian'])

0.0

<details><summary>What is the Gini impurity of a node when we have two classes, each with two items?</summary>
    
- Our Gini impurity is 0.5.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
&=& 1 - \left(\frac{1}{2}\right)^2 - \left(\frac{1}{2}\right)^2 \\
&=& 1 - \frac{1}{4} - \frac{1}{4} \\
&=& \frac{1}{2}
\end{eqnarray*}
$$
</details>

In [5]:
# What is Gini when we have two classes, each with two items?
gini(['Indian', 'Indian', 'Mexican', 'Mexican'])

0.5

<details><summary>What is the Gini impurity of a node when we have two classes, each with three items?</summary>
    
- Our Gini impurity is 0.5.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
&=& 1 - \left(\frac{1}{2}\right)^2 - \left(\frac{1}{2}\right)^2 \\
&=& 1 - \frac{1}{4} - \frac{1}{4} \\
&=& \frac{1}{2}
\end{eqnarray*}
$$
</details>

In [6]:
# What is Gini when we have two classes, each with three items?
gini(['Indian', 'Indian', 'Mexican', 'Mexican', 'Indian', 'Mexican'])

0.5

<details><summary>What is the Gini impurity of a node when we have three classes, each with two items?</summary>
    
- Our Gini impurity is 0.6667.

$$
\begin{eqnarray*}
\text{Gini impurity} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
&=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 - P(\text{class 3})^2 \\
&=& 1 - \left(\frac{1}{3}\right)^2 - \left(\frac{1}{3}\right)^2 - \left(\frac{1}{3}\right)^2 \\
&=& 1 - \frac{1}{9} - \frac{1}{9} - \frac{1}{9} \\
&=& 1 - \frac{1}{3} \\
&=& \frac{2}{3}
\end{eqnarray*}
$$

In [7]:
# What is Gini when we have three classes, each with two items?
gini(['Indian', 'Indian', 'Mexican', 'Mexican', 'Sushi', 'Sushi'])

0.6666666666666667

<details><summary>Summary of Gini Impurity Scores</summary>

- In the binary case, Gini impurity ranges from 0 to 0.5.
- If we have three classes, Gini impurity ranges from 0 to 0.66667.
- If we have $k$ classes, Gini impurity ranges from 0 to $1-\frac{1}{k}$.
- In all cases, a Gini impurity of 0 means maximum purity - all of our observations are from the same class!
</details>

### So how does a decision tree use Gini to decide which variable to split on?

- At any node, consider the subset of our dataframe that exists at that node.
- Iterate through each variable that could potentially split the data.
- Calculate the Gini impurity for every possible split.
- Select the variable that causes the greatest decrease in Gini impurity from the parent node to the child node.

One consequence of this is that a decision tree is fit using a **greedy** algorithm. Simply put, a decision tree makes the best short-term decision by optimizing at each node individually. _This might mean that our tree isn't optimal in the long run!_

## Building a Decision Tree

In [7]:
# Read in Titanic data.
titanic = pd.read_csv('../titanic_clean.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [6]:
# Import data.
import pandas as pd

# Read in Titanic data.
titanic = pd.read_csv('../titanic_clean.csv')

# Change sex to float.
titanic['Sex'] = titanic['Sex'].map({'male':0,
                                     'female':1})

# Create embarked_S column.
titanic['Embarked_s'] = titanic['Embarked'].map({'S':1,
                                                 'C':0,
                                                 'Q':0})

# Create embarked_C column.
titanic['Embarked_c'] = titanic['Embarked'].map({'S':0,
                                                 'C':1,
                                                 'Q':0})

# Conduct train/test split.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(titanic.drop(['Survived','PassengerId','Name','Embarked'], axis=1),
                                                    titanic['Survived'],
                                                    test_size = 0.3,
                                                    random_state = 42)

In [8]:
# Check out first five rows of X_train.
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_s,Embarked_c
43,2,1,5.0,1,2,27.7500,1,0
42,3,0,28.5,0,0,7.2292,0,1
73,1,0,71.0,0,0,34.6542,0,1
167,3,0,24.0,0,0,7.0500,1,0
449,3,0,32.0,0,0,7.8542,1,0


In [12]:
# Import model.
from sklearn.tree import DecisionTreeClassifier

In [13]:
# Instantiate model.
dt = DecisionTreeClassifier(random_state = 42)

In [14]:
# Fit model.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [15]:
# Evaluate model.
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.9939759036144579
Score on testing set: 0.7009345794392523


<details><summary>What conclusion would you make here?</summary>

- Our model is **very** overfit to the data.
</details>

When fitting a decision tree, your model will always grow until it nearly perfectly predicts every observation!
- This is like playing a game of 20 questions, but instead calling it "Infinite Questions." You're always going to be able to win!

<details><summary>Intuitively, what might you try to do to solve this problem?</summary>
    
- As with all models, try to gather more data.
- As with all models, remove some features.
- Is there a way for us to stop our model from growing? (Yes!)
</details>

### Hyperparameters of Decision Trees
There are two hyperparameters of decision trees that we may commonly tune in order to prevent overfitting.

- `max_depth`: The maximum depth of the tree.
    - By default, the nodes are expanded until all leaves are pure (or some other argument limits the growth of the tree).
    - In the 20 questions analogy, this is like "How many questions we can ask?"
- `min_samples_split`: The minimum number of samples required to split an internal node.
    - By default, the minimum number of samples required to split is 2. That is, if there are two observations in a node and if we haven't already achieved maximum purity, we can split it!
- `min_samples_leaf`: The minimum number of samples required to be in a leaf node (a terminal node at the end of the tree).
    - By default, the minimum nubmer of samples required in a leaf node is 1. (This should ring alarm bells - it's very possible that we'll overfit our model to the data!)

[Source: Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [16]:
# Instantiate model with:
# - a maximum depth of 5.
# - at least 7 samples required in order to split an internal node.
# - at least 3 samples in each leaf node.
# - random state of 42.

dt = DecisionTreeClassifier(max_depth = 5,
                            min_samples_split = 7,
                            min_samples_leaf = 3,
                            random_state = 42)

In [17]:
# Fit model.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [18]:
# Evaluate model.
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.8714859437751004
Score on testing set: 0.7897196261682243


#### Let's GridSearch to try to find the best tree.

- Check [3, 5, 7, 10] for `max_depth`.
- Check [5, 10, 15, 20] for `min_samples_split`.
- Check [2, 3, 4, 5, 6, 7] for `min_samples_leaf`.
- Run 5-fold cross-validation.

<details><summary>How many models are being fit here?</summary>

- 4 * 4 * 6 * 5 = 480 models.
</details>

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
grid = GridSearchCV(estimator = DecisionTreeClassifier(),
                    param_grid = {'max_depth': [3, 5, 7, 10],
                                  'min_samples_split': [5, 10, 15, 20],
                                  'min_samples_leaf': [2, 3, 4, 5, 6, 7]},
                    cv = 5,
                    verbose = 1)

In [21]:
import time

# Start our timer.
t0 = time.time()

# Let's GridSearch over the above parameters on our training data.
grid.fit(X_train, y_train)

# Stop our timer and print the result.
print(time.time() - t0)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


2.4993460178375244


[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:    2.5s finished
/Users/gateam/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [22]:
# What is our best decision tree?
grid.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [23]:
# What were the best parameters of our decision tree?
grid.best_params_

{'max_depth': 7, 'min_samples_leaf': 6, 'min_samples_split': 15}

In [24]:
# What was the cross-validated score of the above decision tree?
grid.best_score_

0.8172690763052208

In [25]:
# Instantiate model with best parameters.
dt = DecisionTreeClassifier(max_depth = 7,
                            min_samples_split = 15,
                            min_samples_leaf = 6,
                            random_state = 42)

# Fit model.
dt.fit(X_train, y_train)

# Evaluate model.
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.8775100401606426
Score on testing set: 0.7523364485981309


In [26]:
# Generate predictions on test set.
preds = dt.predict(X_test)

In [27]:
# Import confusion_matrix.
from sklearn.metrics import confusion_matrix

In [28]:
# Generate confusion matrix.
tn, fp, fn, tp = confusion_matrix(y_test,
                                  preds).ravel()

print(confusion_matrix(y_test,
                       preds))

[[101  21]
 [ 32  60]]


In [29]:
# Calculate sensitivity.

sens = tp / (tp + fn)

print(f'Sensitivity: {round(sens, 4)}')

Sensitivity: 0.6522


In [30]:
# Calculate specificity.

spec = tn / (tn + fp)

print(f'Specificity: {round(spec, 4)}')

Specificity: 0.8279


## What's so great about decision trees?


### 1. We don't have to scale our data.
Much in the same way that decision trees don't need much preprocessing in order to work, the base assumptions around how data is used and estimated, don't rely on scale to be effective.

### 2. Decision trees don't care about how your data is distributed.

Is your data heavily skewed or not normally distributed? Decision trees are nonparametric, meaning we don't make assumptions about how our data or errors are distributed.

### 3. Easy to interpret.

The output of a decision tree is easy to interpet and thus relatable to non-technical people. (We'll talk about `feature_importance` later.)

### 4. Speed.

Decision trees are fit very quickly!

> **Protip**
>
> Consider creating a benchmark using a decision tree to understand how one might behave on its own before using a more complicated method.  Do your best to understand how a simple model behaves on a set of data, before using a more complex model. This is a great practice to get into.

## What are the downsides of decision trees?


### 1. Decision trees can very easily overfit.
Decision trees often suffer from high error due to variance, so we need to take special care to avoid this. (Luckily, there are lots of algorithms designed to do exactly this!)

### 2. Decision trees are locally optimal.
Because we're making the best decision at each node (greedy), we might end up with a worse solution in the long run.

### 3. Decision trees don't work well with unbalanced data.
We often will bias our results toward the majority class. We need to take steps to avoid this as well! (Check out the `class_weight` parameter if you're interested.)